<a href="https://colab.research.google.com/github/Kaushal-01/Emotional-Analysis-Chatbot/blob/main/bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets evaluate accelerate torch flask-ngrok spotipy sqlalchemy pyngrok
!pip install -q flask

from IPython.display import clear_output; clear_output()
print("✅ All dependencies installed!")

✅ All dependencies installed!


In [ ]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 98.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:
      Successfully uninstalled transformers-4.56.2


In [ ]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

#GoEmotions
goemotions = load_dataset("go_emotions", "simplified")

mapping = {
    "joy": "joy_happiness", "excitement": "joy_happiness", "approval": "joy_happiness",
    "sadness": "sadness", "disappointment": "sadness", "grief": "sadness",
    "relief": "relaxation_calm", "contentment": "relaxation_calm",
    "pride": "triumph", "gratitude": "triumph",
    "amusement": "amusement", "curiosity": "amusement", "realization": "amusement",
    "nervousness": "anxiety", "embarrassment": "anxiety",
    "fear": "fear_scared", "remorse": "fear_scared",
    "desire": "eroticism", "love": "eroticism", "caring": "eroticism",
    "nostalgia": "nostalgia",
    "admiration": "dreaminess", "surprise": "dreaminess",
    "anger": "anger", "disgust": "anger", "annoyance": "anger"
}

label_names = goemotions["train"].features["labels"].feature.names

def map_labels(example):
    mapped = []
    for l in example["labels"]:
        lbl = label_names[l]
        if lbl in mapping:
            mapped.append(mapping[lbl])
    return {"mapped_labels": list(set(mapped))}

goemotions = goemotions.map(map_labels)

def flatten(example):
    if example["mapped_labels"]:
        return {"text": example["text"], "label": example["mapped_labels"][0]}
    return {"text": None, "label": None}

flat = goemotions.map(flatten)
df = pd.DataFrame(flat["train"]).dropna()

target_labels = sorted(set(mapping.values()))
existing_labels = [lbl for lbl in target_labels if lbl in df["label"].unique()]

min_count = df["label"].value_counts()[existing_labels].min()
balanced = pd.DataFrame(columns=["text","label"])

for lbl in existing_labels:
    subset = df[df["label"] == lbl]
    subset = subset.sample(min_count, random_state=42)
    balanced = pd.concat([balanced, subset])

balanced = balanced.sample(frac=1, random_state=42).reset_index(drop=True)
train, temp = train_test_split(balanced, test_size=0.2, stratify=balanced["label"], random_state=42)
val, test = train_test_split(temp, test_size=0.5, stratify=temp["label"], random_state=42)

# full list
full_label_list = sorted(balanced["label"].unique().tolist())

train.to_csv("train.csv", index=False)
val.to_csv("validation.csv", index=False)
test.to_csv("test.csv", index=False)

print("Train/Val/Test sizes:", len(train), len(val), len(test))
display(train.head())
print("Full label list:", full_label_list)

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

Train/Val/Test sizes: 1072 134 134


,text,label,labels,id,mapped_labels
325,You gotta watch with subtitles IMO. The dubbed...,sadness,[9],edg3atf,[sadness]
43,"[NAME] no, these are horrid on anyone.",fear_scared,[14],ed5m6ii,[fear_scared]
550,B + A? Is that [NAME] and [NAME]?,amusement,"[7, 27]",ednmyfg,[amusement]
30,Well [NAME] just missed...so there’s that bit ...,relaxation_calm,"[17, 23]",edesq5w,"[relaxation_calm, joy_happiness]"
520,Depends on the drug. Weed makes me antisocial ...,anxiety,[19],ee0yl7a,[anxiety]


Full label list: ['amusement', 'anger', 'anxiety', 'dreaminess', 'eroticism', 'fear_scared', 'joy_happiness', 'relaxation_calm', 'sadness', 'triumph']


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import evaluate
import pandas as pd
from datasets import load_dataset, ClassLabel, Dataset

label_list = full_label_list


dataset = load_dataset("csv", data_files={
    "train": "train.csv",
    "validation": "validation.csv",
    "test": "test.csv"
})

label_feature = ClassLabel(names=label_list)

dataset = dataset.cast_column("label", label_feature)


MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Batched preprocessing
def preprocess(batch):
    enc = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)
    enc["labels"] = batch["label"]
    return enc


tokenized = dataset.map(preprocess, batched=True, remove_columns=["text", "label"])


model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(label_list))

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    acc = accuracy.compute(predictions=preds, references=p.label_ids)["accuracy"]
    f1s = f1.compute(predictions=preds, references=p.label_ids, average="weighted")["f1"]
    return {"accuracy": acc, "f1_weighted": f1s}

args = TrainingArguments(
    output_dir="./emotion_model",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    logging_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model("./emotion_model")
tokenizer.save_pretrained("./emotion_model")
print("✅ Model and tokenizer saved in ./emotion_model")

Map:   0%|          | 0/1072 [00:00<?, ? examples/s]

Map:   0%|          | 0/134 [00:00<?, ? examples/s]

Map:   0%|          | 0/134 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2902628647.py:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted
1,2.248900,1.859461,0.425373,0.413671
2,1.795000,1.426237,0.552239,0.540060
3,1.179600,1.316745,0.589552,0.569972


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


✅ Model and tokenizer saved in ./emotion_model


In [ ]:
import json
from transformers import pipeline
import torch
import random

# --- Dummy Content for 11 Emotions (MUST BE EXPANDED) ---
quotes = {
    "joy_happiness": ["Happiness depends upon ourselves. — Aristotle", "Count your age by friends, not years. — John Lennon"],
    "sadness": ["Tears come from the heart, not the brain. — Leonardo da Vinci", "The soul usually knows what to do to heal itself."],
    "relaxation_calm": ["Adopt the pace of nature: her secret is patience. — Emerson", "Breathe. Let go. And remind yourself that this very moment is the only one you know you have."],
    "triumph": ["The only way to do great work is to love what you do. — Steve Jobs", "Success is not final, failure is not fatal."],
    "amusement": ["A day without laughter is a day wasted. — Charlie Chaplin", "Laughter is the shortest distance between two people."],
    "anxiety": ["You are stronger than you think. You have survived every bad day so far.", "Worrying does not empty tomorrow of its troubles, it empties today of its strength."],
    "fear_scared": ["Courage is not the absence of fear, but the triumph over it. — Nelson Mandela", "We can't be brave without being scared."],
    "dreaminess": ["The future belongs to those who believe in the beauty of their dreams. — Eleanor Roosevelt", "Life is what happens when you're busy making other plans."],
    "eroticism": ["Love is the ultimate expression of the will to live. — Tom Wolfe", "The best thing to hold onto in life is each other."],
    "nostalgia": ["Memories warm you up from the inside. — Bob Dylan", "I'm homesick for a place I'm not even sure exists."],
    "anger": ["Speak when you are angry and you will make the best speech you will ever regret. — Ambrose Bierce", "Holding onto anger is like drinking poison and expecting the other person to die."]
}

trivia = {
    "joy_happiness": ["Listening to music you love releases dopamine in your brain."],
    "sadness": ["The term 'melancholy' comes from the Greek for 'black bile'."],
    "relaxation_calm": ["Listening to Lo-fi music can lower heart rate and blood pressure."],
    "triumph": ["Victory gardens were popular during the World Wars to boost morale."],
    "amusement": ["A chuckle is a deep, full, quiet, breathy laugh."],
    "anxiety": ["Taking a short, sharp walk can reduce immediate anxious feelings."],
    "fear_scared": ["The word 'phobia' comes from the Greek word 'phobos', meaning fear or terror."],
    "dreaminess": ["Studies show that creative daydreaming can improve problem-solving."],
    "eroticism": ["The scent of vanilla is often associated with relaxation and mood-lifting."],
    "nostalgia": ["Vinyl records are making a comeback."],
    "anger": ["Counting backwards from ten can disrupt the emotional escalation of anger."]
}

with open("quotes.json", "w") as f: json.dump(quotes, f, indent=2)
with open("trivia.json", "w") as f: json.dump(trivia, f, indent=2)

with open("quotes.json") as f: quotes = json.load(f)
with open("trivia.json") as f: trivia = json.load(f)

print("Loading GPT-2 paraphraser... (This might take a minute)")
paraphraser = pipeline("text-generation", model="gpt2", device=0 if torch.cuda.is_available() else -1)

def paraphrase_text(text):
    try:

        prompt = f"Paraphrase the following quote: '{text}'"
        outputs = paraphraser(
            prompt,
            max_length=len(prompt.split()) + random.randint(5, 15),
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            pad_token_id=paraphraser.tokenizer.eos_token_id
        )

        result = outputs[0]["generated_text"].strip()

        if result.startswith(prompt):
            result = result[len(prompt):].strip()


        return result.split('\n')[0].strip('"')
    except Exception as e:
        # print(f"Paraphrase error: {e}") # Debugging
        return

Loading GPT-2 paraphraser... (This might take a minute)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
import sqlite3, time, random
from datetime import datetime, timedelta
from flask import Flask, request, jsonify, render_template_string
import torch, torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import math


# Spotify Config
SPOTIPY_CLIENT_ID = "d4a6532f7ff047e0872726e89e68dd21"
SPOTIPY_CLIENT_SECRET = "fea62be70e464d558ed6d9366871c247"

if SPOTIPY_CLIENT_ID == "YOUR_SPOTIFY_CLIENT_ID":
    print("🚨 WARNING: Please replace 'YOUR_SPOTIFY_CLIENT_ID' and 'YOUR_SPOTIFY_CLIENT_SECRET' in the code!")
    sp = None
else:
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET
    ))

# Load Model
tokenizer = AutoTokenizer.from_pretrained("./emotion_model")
model = AutoModelForSequenceClassification.from_pretrained("./emotion_model")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

label_list = [
    "joy_happiness","sadness","relaxation_calm","triumph","amusement",
    "anxiety","fear_scared","dreaminess","eroticism","nostalgia","anger"
]
id2label = {i: l for i, l in enumerate(label_list)}

try:
    with open("quotes.json") as f: quotes = json.load(f)
    with open("trivia.json") as f: trivia = json.load(f)
except:
    print("🚨 ERROR: quotes.json or trivia.json not found. Using default data.")
    quotes = {"default": ["Hello!"], "joy_happiness": ["Great day."]}
    trivia = {"default": ["Fun fact."], "joy_happiness": ["Sun is big."]}


# DB Memory
DB_PATH="chatbot_memory.db"
def init_db():
    conn=sqlite3.connect(DB_PATH); cur=conn.cursor()

    cur.execute("""CREATE TABLE IF NOT EXISTS events (user TEXT,text TEXT,emotion TEXT,score REAL,ts INTEGER)""")
    conn.commit(); conn.close()
init_db()

def save_event(user,text,emotion,score):
    conn=sqlite3.connect(DB_PATH); cur=conn.cursor()
    cur.execute("INSERT INTO events VALUES (?,?,?,?,?)",(user,text,emotion,score,int(time.time())))
    conn.commit(); conn.close()

def get_events(user,hours=48):

    cutoff=int((datetime.utcnow()-timedelta(hours=hours)).timestamp())
    conn=sqlite3.connect(DB_PATH); cur=conn.cursor()
    cur.execute("SELECT emotion,score,ts FROM events WHERE user=? AND ts>=? ORDER BY ts DESC",(user,cutoff))
    rows=cur.fetchall(); conn.close()
    return rows

def dominant_emotion(events):
    if not events: return None
    weights={}; now=time.time()

    HALF_LIFE_SEC = 24 * 3600
    # Decay constant lambda
    LAMBDA = math.log(2) / HALF_LIFE_SEC

    for e, s, t in events:
        time_diff_sec = now - t
        # Exponential decay model: W = e^(-lambda * t_diff)
        recency_weight = math.exp(-LAMBDA * time_diff_sec)

        # Combine detection score (s) and recency weight
        weights[e] = weights.get(e, 0) + s * recency_weight

    return max(weights, key=weights.get)

# Prediction
def predict(text):
    inputs=tokenizer(text,return_tensors="pt",truncation=True,padding=True).to(device)
    with torch.no_grad(): logits=model(**inputs).logits
    probs=F.softmax(logits,dim=-1)[0]
    idx=int(probs.argmax())
    return id2label[idx], float(probs[idx])

# Spotify queries mapping
emo_query={
    "joy_happiness":"genre:pop","sadness":"genre:acoustic","anger":"genre:metal",
    "nostalgia":"genre:classic rock","relaxation_calm":"genre:jazz","fear_scared":"genre:dark ambient",
    "anxiety":"genre:lofi","triumph":"genre:epic","dreaminess":"genre:dream pop",
    "eroticism":"genre:rnb","amusement":"genre:comedy"
}

def spotify_tracks(emotion):
    if sp is None:
        return ["Spotify API not configured.", "Add your credentials in Cell 5!"]
    try:
        query = emo_query.get(emotion, "indie")
        res=sp.search(q=query, limit=3, type="track")

        return [f"{t['name']} — {t['artists'][0]['name']}" for t in res["tracks"]["items"]]
    except Exception as e:
        print(f"Spotify Error: {e}")
        return [f"Couldn't load music for {emotion} right now.", "Try again soon!"]

# UI colors
emotion_colors={
    "joy_happiness":"#fff9c4","sadness":"#cce7ff","anger":"#ffcdd2",
    "nostalgia":"#f5e1c0","relaxation_calm":"#dcedc8","fear_scared":"#e1bee7",
    "anxiety":"#ffe0b2","triumph":"#c8e6c9","dreaminess":"#d1c4e9",
    "eroticism":"#f8bbd0","amusement":"#ffecb3"
}

# 3. FLASK APP
app=Flask(__name__)

html_template="""
<!DOCTYPE html>
<html lang="en">
<head>
    <title>🎶 Serenity Chatbot</title>
    <style>
        body { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; transition: background-color 0.8s ease; margin: 0; display: flex; justify-content: center; align-items: center; min-height: 100vh; background-color: #f5f7fa; }
        .chat-container { width: 100%; max-width: 600px; background: white; box-shadow: 0 4px 12px rgba(0,0,0,0.1); border-radius: 10px; overflow: hidden; display: flex; flex-direction: column; height: 80vh; max-height: 900px; }
        .header { background-color: #5d6d7e; color: white; padding: 15px; text-align: center; font-size: 1.2em; }
        #chat-box { flex-grow: 1; padding: 20px; overflow-y: auto; background-color: #f9f9f9; }
        .message-row { display: flex; margin-bottom: 15px; }
        .user-message { justify-content: flex-end; }
        .bot-message { justify-content: flex-start; }
        .bubble { max-width: 70%; padding: 12px 18px; border-radius: 20px; line-height: 1.4; word-wrap: break-word; }
        .user-message .bubble { background-color: #007bff; color: white; border-bottom-right-radius: 5px; }
        .bot-message .bubble { background-color: #e0e0e0; color: #333; border-bottom-left-radius: 5px; }
        .input-area { display: flex; padding: 15px; border-top: 1px solid #ddd; background: white; }
        #user-input { flex-grow: 1; padding: 10px 15px; border: 1px solid #ccc; border-radius: 20px; font-size: 16px; outline: none; }
        #send-button { margin-left: 10px; background-color: #5d6d7e; color: white; border: none; padding: 10px 15px; border-radius: 20px; cursor: pointer; transition: background-color 0.3s; }
        #send-button:hover { background-color: #4a5967; }
        .songs-list { font-size: 0.9em; margin-top: 10px; padding-top: 5px; border-top: 1px dashed #aaa; }
    </style>
</head>
<body>
    <div class="chat-container">
        <div class="header" id="header">🎶 Serenity (calm)</div>
        <div id="chat-box">
            <div class="message-row bot-message">
                <div class="bubble">Hello! I'm Serenity. Tell me how you're feeling today.</div>
            </div>
        </div>
        <div class="input-area">
            <input type="text" id="user-input" placeholder="Type your message...">
            <button id="send-button">Send</button>
        </div>
    </div>

    <script>
        const chatBox = document.getElementById('chat-box');
        const userInput = document.getElementById('user-input');
        const sendButton = document.getElementById('send-button');
        const header = document.getElementById('header');

        function appendMessage(text, isUser, emotion) {
            const row = document.createElement('div');
            row.className = 'message-row ' + (isUser ? 'user-message' : 'bot-message');

            const bubble = document.createElement('div');
            bubble.className = 'bubble';

            if (!isUser) {
                // Split the response for formatting songs
                const parts = text.split(" Songs: ");
                const mainMessage = parts[0];
                bubble.innerHTML = mainMessage + ".";

                if (parts.length > 1) {
                    const songsList = document.createElement('div');
                    songsList.className = 'songs-list';
                    songsList.innerHTML = '✨ Music Vibe: ' + parts[1].replace(/, /g, '<br>');
                    bubble.appendChild(songsList);
                }

                // Update UI based on detected emotion
                if (emotion) {
                    const cleanEmotion = emotion.replace('_', ' ');
                    document.body.style.backgroundColor = emotion_colors[emotion] || '#f5f7fa';
                    header.textContent = `🎶 Serenity (${cleanEmotion})`;
                }
            } else {
                 bubble.textContent = text;
            }

            row.appendChild(bubble);
            chatBox.appendChild(row);
            chatBox.scrollTop = chatBox.scrollHeight; // Scroll to bottom
        }

        const emotion_colors = JSON.parse('{{ emotion_colors | tojson | safe }}');

        async function sendMessage() {
            const text = userInput.value.trim();
            if (text === "") return;

            // 1. Display User Message
            appendMessage(text, true);
            userInput.value = "";
            sendButton.disabled = true;

            try {
                // 2. Send to Flask API
                const response = await fetch('/chat', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    // Using a hardcoded user ID for Colab deployment
                    body: JSON.stringify({ user: 'colab_user_1', text: text })
                });

                const data = await response.json();

                // 3. Display Bot Response
                appendMessage(data.message, false, data.emotion);
            } catch (error) {
                console.error("API Error:", error);
                appendMessage("Sorry, I ran into an error. Please try again or check the backend console.", false, 'sadness');
            }
            sendButton.disabled = false;
        }

        sendButton.addEventListener('click', sendMessage);
        userInput.addEventListener('keypress', (e) => {
            if (e.key === 'Enter') sendMessage();
        });
    </script>
</body>
</html>
"""

# Route
@app.route("/chat",methods=["POST"])
def chat():
    try:
        data=request.get_json()
        user=data.get("user","anon"); text=data.get("text","")

        # 1. Predict Emotion
        emotion,score=predict(text)

        # 2. Save Event
        save_event(user,text,emotion,score)

        # 3. Emotional Memory
        dom=dominant_emotion(get_events(user))

        # 4. Dynamic Content
        songs=spotify_tracks(emotion)

        content_type = random.choice(["quote", "trivia"])
        source_data = quotes if content_type == "quote" else trivia

        # random piece of text
        raw_piece = random.choice(source_data.get(emotion, source_data.get("joy_happiness", ["Hello!"])))

        # Paraphrase
        piece = paraphrase_text(raw_piece) if random.random() < 0.7 else raw_piece

        display_emotion = emotion.replace('_',' ')

        # Base Message
        msg = f"You seem to be feeling {display_emotion}. {piece} Songs: {', '.join(songs)}."

        if dom and dom != emotion:
             msg += f" Over the last two days, you've mostly been feeling {dom.replace('_',' ')}."

        return jsonify({
            "emotion": emotion,
            "message": msg,
            "color": emotion_colors.get(emotion,"#f5f7fa") # Fallback color
        })
    except Exception as e:
        print(f"Backend Chat Error: {e}")
        return jsonify({
            "emotion": "sadness",
            "message": "Oops! I hit a snag on my end. I'm taking a moment to calm down.",
            "color": emotion_colors.get("sadness")
        }), 500

@app.route("/")
def index():
    return render_template_string(html_template, emotion_colors=emotion_colors)

print("Flask app defined. Ready to run in the next cell.")

Flask app defined. Ready to run in the next cell.


In [ ]:
from pyngrok import ngrok
from threading import Thread
import os
from google.colab import userdata

try:
    NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    if not NGROK_AUTH_TOKEN:
        raise ValueError("Secret not found or empty.")

    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("✅ ngrok Authtoken successfully configured from Colab Secrets.")

except Exception as e:
    print(f"🚨 FAILED to get Authtoken from Colab Secrets: {e}")
    print("Please verify you created a Secret named 'NGROK_AUTH_TOKEN' in the sidebar.")


ngrok.kill()

try:
    http_tunnel = ngrok.connect(5000, "http")
    public_url = http_tunnel.public_url

    print(f"\n🌐 Serenity Chatbot UI: {public_url}")


    thread = Thread(target=app.run, kwargs={"use_reloader": False, "host": "0.0.0.0", "port": 5000})
    thread.start()

    print("\nNote: Access the chatbot via the URL printed above.")

except Exception as e:
    print(f"\n🚨 ERROR connecting ngrok tunnel or starting Flask app. Details: {e}")
    print("Make sure your Flask app in the previous cell is named 'app' and is ready to run on port 5000.")

✅ ngrok Authtoken successfully configured from Colab Secrets.

🌐 Serenity Chatbot UI: https://uncavernous-guadalupe-concedingly.ngrok-free.dev

Note: Access the chatbot via the URL printed above.


In [ ]:
import requests
import time

time.sleep(5)

NGROK_URL = public_url # Variable from Cell 6

print(f"Testing API endpoint: {NGROK_URL}/chat")

try:
    test_user_id = "test_user_" + str(int(time.time()))

    resp1 = requests.post(f"{NGROK_URL}/chat", json={"user": test_user_id, "text": "I feel very nostalgic today. I miss my childhood."})

    print("\n--- Test 1 (Nostalgia) ---")
    print("Status:", resp1.status_code)
    print("Response:", resp1.json())

    time.sleep(2)
    resp2 = requests.post(f"{NGROK_URL}/chat", json={"user": test_user_id, "text": "But you know, I did get a great new job today!"})

    print("\n--- Test 2 (Triumph/Joy + Memory) ---")
    print("Status:", resp2.status_code)

    print("Response:", resp2.json())

except Exception as e:
    print(f"\n🚨 TEST FAILED. Ensure Cell 6 is running and the ngrok URL is active. Error: {e}")

Testing API endpoint: https://uncavernous-guadalupe-concedingly.ngrok-free.dev/chat


/tmp/ipython-input-4034336721.py:69: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  cutoff=int((datetime.utcnow()-timedelta(hours=hours)).timestamp())
INFO:werkzeug:127.0.0.1 - - [04/Oct/2025 15:57:21] "POST /chat HTTP/1.1" 200 -



--- Test 1 (Nostalgia) ---
Status: 200
Response: {'color': '#ffe0b2', 'emotion': 'anxiety', 'message': 'You seem to be feeling anxiety. You are stronger than you think. You have survived every bad day so far. Songs: .'}


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=26) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
INFO:werkzeug:127.0.0.1 - - [04/Oct/2025 15:57:42] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2025 15:57:43] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2025 15:58:02] "POST /chat HTTP/1.1" 200 -



--- Test 2 (Triumph/Joy + Memory) ---
Status: 200
Response: {'color': '#c8e6c9', 'emotion': 'triumph', 'message': "You seem to be feeling triumph. 'You're not going to beat yourself up about the way things work. If you're going to love what you do, you have to love it. You have to love what you love.' Songs: Pinch A Twitch — Songs To Your Eyes, Force — FineTune Music, Tell No Lies — Songs To Your Eyes."}
